#####Libraries
***

In [ ]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-09-16 09:48:46--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-16 09:48:46--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://cfhcable.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-16 09:48:46--  http://cfhc

#####Load Dataset
***

In [ ]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
  #άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-05-29','2022-09-25')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-05-29','2022-09-25')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,106.605820,81.597748,158.413422,136.604294,136.802231,256.874237,160.932495,203.371292,218.499512,161.456635,...,102.610245,259.269989,252.285995,33.223022,120.749817,398.641052,339.450012,224.728775,169.534683,56.981373
2021-06-02,105.836273,82.125130,156.835663,135.604462,137.594055,257.275818,161.699493,205.369492,215.377518,161.811569,...,102.156342,262.170013,249.654388,33.380482,118.805550,397.788330,338.470001,227.733353,171.020279,57.433758
2021-06-03,106.605820,82.422363,159.030807,135.212372,134.990829,258.382477,159.350494,205.707352,215.249908,162.205963,...,102.345467,257.790009,249.566986,32.563683,117.322823,399.856415,336.119995,226.196365,170.178452,57.659950
2021-06-04,106.748337,82.489479,159.834396,136.084778,138.425522,259.939758,160.311005,204.404190,216.300385,162.679199,...,102.544052,263.040009,251.868439,32.662090,117.313011,397.582520,332.070007,228.209320,170.990570,57.914417
2021-06-07,107.365868,82.144295,159.295410,135.271179,137.554459,259.401062,159.900497,204.780640,212.471527,163.586258,...,102.194168,260.600006,250.334152,32.278294,117.347458,392.299591,316.019989,229.379425,166.187210,57.537434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,141.419998,104.709999,163.699997,99.200691,96.510002,286.609985,133.270004,214.460007,191.050003,158.440002,...,98.480003,96.230003,282.690002,36.119999,93.510002,524.340027,311.140015,205.199997,191.720367,96.500000
2022-09-12,142.240005,105.180000,163.490005,100.773888,96.300003,289.239990,136.449997,214.220001,192.050003,162.449997,...,99.760002,97.650002,282.829987,36.259998,94.980003,531.250000,313.399994,206.630005,192.509048,97.610001
2022-09-13,138.529999,102.199997,158.490005,95.476807,90.389999,277.720001,126.820000,207.850006,184.669998,155.350006,...,95.790001,95.010002,279.480011,35.930000,89.860001,513.960022,294.670013,199.669998,185.940002,95.330002


In [ ]:
rd1= rd.pct_change()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,-0.007219,0.006463,-0.009960,-0.007319,0.005788,0.001563,0.004766,0.009825,-0.014288,0.002198,...,-0.004424,0.011185,-0.010431,0.004739,-0.016102,-0.002139,-0.002887,0.013370,0.008763,0.007939
2021-06-03,0.007271,0.003619,0.013996,-0.002891,-0.018920,0.004301,-0.014527,0.001645,-0.000592,0.002437,...,0.001851,-0.016707,-0.000350,-0.024469,-0.012480,0.005199,-0.006943,-0.006749,-0.004922,0.003938
2021-06-04,0.001337,0.000814,0.005053,0.006452,0.025444,0.006027,0.006028,-0.006335,0.004880,0.002918,...,0.001940,0.020365,0.009222,0.003022,-0.000084,-0.005687,-0.012049,0.008899,0.004772,0.004413
2021-06-07,0.005785,-0.004185,-0.003372,-0.005979,-0.006293,-0.002072,-0.002561,0.001842,-0.017702,0.005576,...,-0.003412,-0.009276,-0.006092,-0.011751,0.000294,-0.013288,-0.048333,0.005127,-0.028091,-0.006509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,0.006405,0.007021,-0.004379,0.015079,0.029001,0.015124,0.026575,0.012416,0.010579,0.032048,...,0.015781,0.000624,-0.009946,-0.003036,0.013549,-0.002890,0.020365,0.022371,0.004919,0.016753
2022-09-12,0.005798,0.004489,-0.001283,0.015859,-0.002176,0.009176,0.023861,-0.001119,0.005234,0.025309,...,0.012998,0.014756,0.000495,0.003876,0.015720,0.013178,0.007264,0.006969,0.004114,0.011503
2022-09-13,-0.026083,-0.028332,-0.030583,-0.052564,-0.061371,-0.039828,-0.070575,-0.029736,-0.038428,-0.043706,...,-0.039796,-0.027035,-0.011844,-0.009101,-0.053906,-0.032546,-0.059764,-0.033683,-0.034123,-0.023358


In [ ]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,0.014382,0.007921,0.004290,0.006533,-0.031250,0.008478,-0.002090,0.008721,0.000143,0.012985,...,0.008100,-0.008508,0.019087,0.011988,0.007984,0.011113,0.007335,0.005688,0.011905,0.002853
2021-06-02,0.007964,-0.000704,-0.002232,-0.001212,0.001522,-0.000279,0.022724,0.003895,-0.001760,0.009345,...,-0.001646,0.004291,-0.017405,0.001445,0.010081,0.011040,-0.005539,0.014799,0.003959,-0.001423
2021-06-03,0.004862,0.000469,0.001981,-0.010569,-0.009989,-0.014865,0.046927,0.010505,-0.016103,0.006291,...,-0.028992,0.008269,-0.015732,0.008367,0.002139,0.002199,-0.025472,0.004065,0.000620,-0.028495
2021-06-04,0.008814,0.004458,0.001020,0.010537,-0.013964,0.007039,0.005670,0.010208,0.014478,0.008375,...,0.002596,0.015343,0.010468,0.018026,-0.019206,0.003412,0.004367,0.001669,0.025224,-0.015643
2021-06-07,-0.003426,-0.003621,-0.016691,-0.017855,-0.017350,-0.030531,0.009422,0.007139,-0.009689,-0.020353,...,-0.032370,-0.007000,-0.031835,-0.016020,-0.019502,-0.004154,-0.021005,-0.014125,-0.004833,-0.004470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#first: ranks assigned in order they appear in the array
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,39,22,16,18,1,26,7,29,9,38,...,24,2,42,36,23,33,21,17,35,13
2021-06-02,35,14,6,12,19,16,45,23,8,37,...,9,25,1,18,39,40,3,42,24,10
2021-06-03,37,28,32,15,16,12,45,43,9,39,...,4,41,10,42,33,34,6,36,30,5
2021-06-04,28,18,9,33,5,22,20,30,38,27,...,13,39,32,43,3,16,17,11,45,4
2021-06-07,35,34,12,9,10,3,44,43,22,6,...,1,27,2,13,8,32,5,16,30,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-09,14,16,2,30,40,31,39,26,23,41,...,33,6,1,3,28,4,35,37,11,34
2022-08-10,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-08-11,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37


In [ ]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,5,3,2,2,1,3,1,4,1,5,...,3,1,5,4,3,4,3,2,4,2
2021-06-02,4,2,1,2,3,2,5,3,1,5,...,1,3,1,2,5,5,1,5,3,2
2021-06-03,5,4,4,2,2,2,5,5,1,5,...,1,5,2,5,4,4,1,4,4,1
2021-06-04,4,2,1,4,1,3,3,4,5,3,...,2,5,4,5,1,2,2,2,5,1
2021-06-07,4,4,2,1,2,1,5,5,3,1,...,1,3,1,2,1,4,1,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-09,2,2,1,4,5,4,5,3,3,5,...,4,1,1,1,4,1,4,5,2,4
2022-08-10,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-08-11,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5


In [ ]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,10,32,5,9,31,24,30,38,3,25,...,12,39,4,29,2,15,14,41,36,35
2021-06-03,37,33,41,15,2,35,4,24,19,30,...,27,3,20,1,5,36,9,11,13,34
2021-06-04,15,13,29,34,45,30,31,4,28,20,...,17,44,36,21,10,5,2,35,27,26
2021-06-07,38,20,23,17,15,28,26,32,3,37,...,22,12,16,8,31,7,1,36,2,14
2021-06-08,8,1,28,32,2,13,44,9,33,23,...,19,36,27,35,34,24,45,22,38,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,14,16,2,30,40,31,39,26,23,41,...,33,6,1,3,28,4,35,37,11,34
2022-09-12,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-09-13,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37


In [ ]:
r5_feature=rd2_feature.copy()
rd3_feature=rank_data(r5_feature)
rd3_feature

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,2,4,1,1,4,3,4,5,1,3,...,2,5,1,4,1,2,2,5,4,4
2021-06-03,5,4,5,2,1,4,1,3,3,4,...,3,1,3,1,1,4,1,2,2,4
2021-06-04,2,2,4,4,5,4,4,1,4,3,...,2,5,4,3,2,1,1,4,3,3
2021-06-07,5,3,3,2,2,4,3,4,1,5,...,3,2,2,1,4,1,1,4,1,2
2021-06-08,1,1,4,4,1,2,5,1,4,3,...,3,4,3,4,4,3,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,2,2,1,4,5,4,5,3,3,5,...,4,1,1,1,4,1,4,5,2,4
2022-09-12,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-09-13,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5


In [ ]:
rd3.loc[rd3.index=='2022-08-19']

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-19,4,4,1,1,1,2,1,3,4,2,...,2,1,2,5,1,5,2,3,4,5


#####Functions
***

In [ ]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [ ]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [ ]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-24] #-23 , -20 ,-2
      
      Y_train= y.iloc[:-24] #-23 , -20

      print(Y_train.tail(2))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(23,0,-1): #from 4/01 and later -->predictions
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      last=X_validation.index
      print('this is last for prediction',last)
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob , first , last 

In [ ]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [ ]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

####**For 29 May-24 June** 

#####Features
***

In [ ]:
stock=dict()
for i in assets:
  stock[i]=pdr.get_data_yahoo(i,'2021-05-29','2022-06-25') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-05-27') & (stock['ABBV'].index <= '2022-06-24')]))
# stock['ABBV']

18


In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-05-27']['Rank'])

Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64
Date
2022-05-27    3.0
Name: Rank, dtype: float64
Date
2022-05-27    2.0
Name: Rank, dtype: float64
Date
2022-05-27    3.0
Name: Rank, dtype: float64
Date
2022-05-27    2.0
Name: Rank, dtype: float64
Date
2022-05-27    1.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    1.0
Name: Rank, dtype: float64
Date
2022-05-27    2.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    2.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    3.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64


#####Logistic Regression (baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-05-18    5
2022-05-19    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.23884975 0.15922462 0.11005146 0.23169092 0.26018326]
Date
2022-05-18    3
2022-05-19    4
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.07452487 0.36369765 0.12824521 0.12967229 0.30385999]
Date
2022-05-18    2
2022-05-19    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.27127617 0.11070039 0.21756218 0.21226199 0.18819927]
Date
2022-05-18    1
2022-05-19    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12316633 0.78280879 0.03331684 0.05213579 0.00857225]
Date
2022-05-18    5
2022-05-19    1
Name: Rank, dtype: int64


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-06-23  150.220001  95.05336  171.254623  102.566132  100.742371   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-23  244.902954  121.139999  197.539993  174.073257  153.789993  ...   

                  PRU       PYPL        RE        ROL       ROST         UNH  \
Date                                                                           
2022-06-23  95.745781  81.650002  272.1651  36.393417  82.340591  527.813843   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-23  277.829987  213.891159  182.465805  89.114151  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-24  150.869995  95.668518  171.234695  102.247513  98.657707   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-24  240.463562  114.809998  197.539993  171.065475  149.830002  ...   

                  PRU       PYPL          RE       ROL       ROST         UNH  \
Date                                                                            
2022-06-24  94.935303  77.040001  270.187073  36.12413  77.686951  529.927246   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-24  276.209991  212.114319  182.705399  88.767509  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
20

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.093184,0.238105,0.309442,0.104123,0.125811,0.155420,0.081323,0.219838,0.099488,0.156470,...,0.085639,0.129017,0.163268,0.263346,0.186684,0.107881,0.083635,0.043021,0.103668,0.332204
2022-05-23,0.155796,0.215327,0.166875,0.108246,0.165830,0.124018,0.092499,0.404843,0.301825,0.121773,...,0.286720,0.324810,0.105847,0.084106,0.025757,0.381384,0.461288,0.141567,0.491085,0.264694
2022-05-24,0.103429,0.195176,0.082412,0.086148,0.096932,0.144780,0.322511,0.215661,0.130058,0.149677,...,0.158469,0.109389,0.037350,0.063368,0.312057,0.105274,0.381810,0.246404,0.088338,0.082659
2022-05-25,0.140874,0.209715,0.271881,0.186632,0.139326,0.085054,0.274159,0.066607,0.083860,0.127824,...,0.237070,0.098078,0.121637,0.124175,0.237407,0.172508,0.155547,0.109151,0.460279,0.098491
2022-05-26,0.135898,0.084411,0.132633,0.118948,0.240882,0.336380,0.108356,0.062228,0.172037,0.210994,...,0.128315,0.159389,0.144679,0.244696,0.123411,0.204917,0.196731,0.113328,0.064838,0.360610
2022-05-27,0.082825,0.165332,0.207081,0.078297,0.127059,0.239197,0.392185,0.054270,0.084838,0.115022,...,0.048619,0.109409,0.235387,0.111886,0.205798,0.070070,0.214152,0.141425,0.073467,0.384408
2022-05-31,0.149528,0.075984,0.068804,0.174772,0.228417,0.433794,0.156416,0.115657,0.072527,0.113231,...,0.111683,0.364135,0.115449,0.289031,0.149702,0.161598,0.144750,0.091485,0.125024,0.110555
2022-06-01,0.115535,0.272927,0.177509,0.126887,0.115039,0.188348,0.182756,0.129628,0.074828,0.088363,...,0.276658,0.367652,0.250153,0.045112,0.255650,0.250373,0.107362,0.119935,0.096894,0.309951
2022-06-02,0.264345,0.290544,0.124891,0.078427,0.098808,0.176497,0.074267,0.115553,0.262469,0.303216,...,0.109720,0.110224,0.148252,0.237978,0.170306,0.273986,0.430527,0.101564,0.138847,0.284781


In [ ]:
print(RPS1.mean(axis=1).mean())

0.16879067749162088


In [ ]:
print(RPS1.mean(axis=1))

date
2022-05-20    0.154587
2022-05-23    0.199091
2022-05-24    0.144556
2022-05-25    0.154686
2022-05-26    0.151327
2022-05-27    0.157827
2022-05-31    0.170798
2022-06-01    0.170169
2022-06-02    0.169571
2022-06-03    0.162950
2022-06-06    0.164417
2022-06-07    0.164820
2022-06-08    0.172709
2022-06-09    0.167502
2022-06-10    0.177788
2022-06-13    0.179786
2022-06-14    0.182984
2022-06-15    0.175017
2022-06-16    0.198987
2022-06-17    0.197753
2022-06-21    0.151719
2022-06-22    0.167151
2022-06-23    0.148062
2022-06-24    0.166720
dtype: float64


In [ ]:
#save results 
RPS1.to_excel('29_May_24_June-log_reg(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('29_May_24_June-PROBABILITIES-log_reg(baseline)-real_data.xlsx')

#####After feature importances 
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-05-18    5
2022-05-19    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15674915 0.17872307 0.17297989 0.21742322 0.27412468]
Date
2022-05-18    3
2022-05-19    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.17831361 0.2330542  0.2069755  0.18369568 0.19796101]
Date
2022-05-18    2
2022-05-19    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-06-23  150.220001  95.05336  171.254623  102.566132  100.742371   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-23  244.902954  121.139999  197.539993  174.073257  153.789993  ...   

                  PRU       PYPL        RE        ROL       ROST         UNH  \
Date                                                                           
2022-06-23  95.745781  81.650002  272.1651  36.393417  82.340591  527.813843   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-23  277.829987  213.891159  182.465805  89.114151  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-24  150.869995  95.668518  171.234695  102.247513  98.657707   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-24  240.463562  114.809998  197.539993  171.065475  149.830002  ...   

                  PRU       PYPL          RE       ROL       ROST         UNH  \
Date                                                                            
2022-06-24  94.935303  77.040001  270.187073  36.12413  77.686951  529.927246   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-24  276.209991  212.114319  182.705399  88.767509  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
20

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.084479,0.237776,0.279687,0.077019,0.134255,0.136062,0.274425,0.233856,0.105046,0.243156,...,0.105322,0.139377,0.127913,0.225508,0.128718,0.093510,0.198356,0.072717,0.278922,0.286939
2022-05-23,0.150775,0.245848,0.121262,0.076475,0.149786,0.100317,0.158299,0.249272,0.286110,0.077377,...,0.283426,0.309835,0.115752,0.080903,0.123068,0.292842,0.245435,0.121537,0.259296,0.164957
2022-05-24,0.095734,0.236572,0.058747,0.111355,0.120939,0.138177,0.187251,0.235689,0.129110,0.240949,...,0.107258,0.134983,0.069620,0.080869,0.215339,0.215276,0.247393,0.241230,0.120586,0.171391
2022-05-25,0.095615,0.236619,0.248817,0.126124,0.136620,0.071159,0.185729,0.115505,0.099317,0.079114,...,0.279895,0.136010,0.115150,0.134465,0.218308,0.220749,0.107265,0.114771,0.258192,0.172655
2022-05-26,0.095169,0.114601,0.120669,0.127438,0.209637,0.273925,0.269864,0.075586,0.133211,0.237501,...,0.108436,0.135330,0.129109,0.225099,0.112835,0.225245,0.197306,0.121912,0.275657,0.286536
2022-05-27,0.086315,0.237447,0.250214,0.076260,0.130965,0.140002,0.193101,0.075706,0.099825,0.130637,...,0.063928,0.128196,0.221569,0.110651,0.216514,0.065592,0.245221,0.114020,0.272290,0.282761
2022-05-31,0.152152,0.114916,0.055914,0.126719,0.207791,0.274421,0.256593,0.122771,0.064297,0.081460,...,0.125844,0.315483,0.069877,0.250701,0.129254,0.129035,0.121074,0.073047,0.057111,0.174317
2022-06-01,0.095297,0.246554,0.121089,0.125654,0.121720,0.138210,0.257206,0.122713,0.064497,0.081345,...,0.240943,0.313530,0.270317,0.079411,0.251205,0.271219,0.142197,0.120600,0.121778,0.279659
2022-06-02,0.292776,0.236141,0.108373,0.076556,0.149458,0.139420,0.154128,0.122061,0.279852,0.233870,...,0.114915,0.133307,0.221791,0.225473,0.116766,0.231227,0.201898,0.073679,0.267447,0.277358


In [ ]:
print(RPS2.mean(axis=1).mean())

0.15931212587151186


In [ ]:
print(RPS2.mean(axis=1))

date
2022-05-20    0.161732
2022-05-23    0.170448
2022-05-24    0.148594
2022-05-25    0.152043
2022-05-26    0.159127
2022-05-27    0.155629
2022-05-31    0.160358
2022-06-01    0.169028
2022-06-02    0.155714
2022-06-03    0.162123
2022-06-06    0.152909
2022-06-07    0.151691
2022-06-08    0.162648
2022-06-09    0.159660
2022-06-10    0.159234
2022-06-13    0.165277
2022-06-14    0.163459
2022-06-15    0.165346
2022-06-16    0.164478
2022-06-17    0.168675
2022-06-21    0.149238
2022-06-22    0.153001
2022-06-23    0.150825
2022-06-24    0.162254
dtype: float64


In [ ]:

#save results 
RPS2.to_excel('29_May_24_June-log_reg(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('29_May_24_June-PROBABILITIES-log_reg(with_feature_imp)-real_data.xlsx')

####**For 26 June.-22 July.**

#####Features
***

In [ ]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-06-26','2022-07-23') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-06-28  113.260002  114.110001  112.839996  113.000000  107.356354
 2021-06-29  112.650002  112.750000  111.610001  112.300003  106.691330
 2021-06-30  112.209999  112.919998  111.629997  112.639999  107.014336
 2021-07-01  112.949997  114.550003  112.889999  114.260002  108.553436
 2021-07-02  114.519997  115.489998  113.889999  115.169998  109.417984
 ...                ...         ...         ...         ...         ...
 2022-07-18  153.800003  154.470001  148.889999  149.570007  149.570007
 2022-07-19  150.000000  150.229996  148.800003  149.740005  149.740005
 2022-07-20  149.529999  150.449997  147.339996  147.690002  147.690002
 2022-07-21  147.240005  147.839996  146.020004  147.750000  147.750000
 2022-07-22  148.470001  149.289993  147.580002  148.470001  148.470001
 
 [270 rows x 5 columns],
 'AEP':                  Open

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-06-26') & (stock['ABBV'].index <= '2022-07-22')]))

19


In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-06-24']['Rank'])

Date
2022-06-24    4.0
Name: Rank, dtype: float64
Date
2022-06-24    5.0
Name: Rank, dtype: float64
Date
2022-06-24    5.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    3.0
Name: Rank, dtype: float64
Date
2022-06-24    1.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    4.0
Name: Rank, dtype: float64
Date
2022-06-24    3.0
Name: Rank, dtype: float64
Date
2022-06-24    3.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    1.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    3.0
Name: Rank, dtype: float64
Date
2022-06-24    1.0
Name: Rank, dtype: float64
Date
2022-06-24    4.0
Name: Rank, dtype: float64
Date
2022-06-24    5.0
Name: Rank, dtype: float64
Date
2022-06-24    5.0
Name: Rank, dtype: float64


#####Logistic Regression (baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-06-15    1
2022-06-16    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18232759 0.10899485 0.2816995  0.19961485 0.22736321]
Date
2022-06-15    1
2022-06-16    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.17081737 0.15786053 0.23544256 0.25466446 0.18121508]
Date
2022-06-15    2
2022-06-16    3
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.14217469 0.14908729 0.23878764 0.22627007 0.24368032]
Date
2022-06-15    5
2022-06-16    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1897972  0.27827343 0.22775049 0.27406545 0.03011343]
Date
2022-06-15    5
2022-06-16    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Date
2022-06-15    3
2022-06-16    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.19600363 0.13673831 0.21204129 0.40169944 0.05351734]
Date
2022-06-15    4
2022-06-16    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.30041035 0.17828454 0.19056899 0.10171938 0.22901674]
Date
2022-06-15    2
2022-06-16    2
Name: Rank, dtype: int64


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-21  141.850006  104.940002  171.145065  105.991302  104.363106   

                  AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                  ...   
2022-07-21  284.73999  138.229996  214.979996  196.25322  162.240005  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-07-21  102.199997  96.559998  279.808899  36.540001  90.691246   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-21  546.604919  316.690002  212.820007  200.236176  94.080002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-22  140.339996  103.410004  164.163818  102.695572  100.552856   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-07-22  276.98999  133.220001  210.330002  193.584045  157.699997  ...   

                  PRU       PYPL         RE        ROL       ROST         UNH  \
Date                                                                            
2022-07-22  99.110001  93.580002  273.01001  36.400002  87.801399  542.866638   

                   URI           V        VRSK        XOM  
Date                                                       
2022-07-22  307.549988  208.610001  197.900055  94.010002  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-06-17,0.180259,0.136980,0.126812,0.352600,0.143036,0.175062,0.276201,0.053058,0.205590,0.093921,...,0.105274,0.190271,0.135892,0.120500,0.195697,0.101034,0.129906,0.109522,0.195715,0.049365
2022-06-21,0.144513,0.471836,0.087128,0.129235,0.244528,0.064324,0.116685,0.260018,0.148561,0.191768,...,0.111737,0.224541,0.300411,0.125536,0.099814,0.093681,0.322260,0.105955,0.057100,0.122955
2022-06-22,0.155327,0.392397,0.177746,0.038934,0.047838,0.047556,0.115155,0.164134,0.101007,0.116290,...,0.330144,0.153733,0.103972,0.122285,0.170764,0.328179,0.106770,0.071242,0.289436,0.571137
2022-06-23,0.313539,0.353537,0.114503,0.124830,0.086739,0.059498,0.116912,0.119669,0.095627,0.230644,...,0.117246,0.186661,0.055994,0.105169,0.201518,0.171055,0.128162,0.088099,0.118843,0.174784
2022-06-24,0.126484,0.227717,0.113261,0.126564,0.272995,0.366655,0.365770,0.131184,0.365482,0.106138,...,0.040681,0.259724,0.275344,0.214376,0.192951,0.059975,0.550871,0.183992,0.144834,0.142622
2022-06-27,0.294517,0.113619,0.106464,0.020018,0.234560,0.156933,0.088206,0.233584,0.313204,0.083114,...,0.070294,0.190214,0.252349,0.270365,0.113199,0.082309,0.498307,0.074790,0.082742,0.100764
2022-06-28,0.281049,0.114980,0.130691,0.185694,0.147542,0.317659,0.385403,0.087123,0.108177,0.303162,...,0.133527,0.110156,0.114269,0.197434,0.157857,0.093719,0.515036,0.088959,0.054069,0.216958
2022-06-29,0.276105,0.205283,0.055090,0.042846,0.190970,0.230045,0.177313,0.126344,0.081584,0.080455,...,0.078750,0.343625,0.306004,0.262538,0.243099,0.224921,0.107889,0.066809,0.190117,0.362638
2022-06-30,0.111273,0.109891,0.242616,0.272066,0.119932,0.150763,0.269804,0.114823,0.095672,0.073447,...,0.282537,0.369611,0.195884,0.104112,0.344104,0.164908,0.075273,0.276556,0.132261,0.114593


In [ ]:
print(RPS1.mean(axis=1).mean())

0.1691415912823646


In [ ]:
print(RPS1.mean(axis=1))

date
2022-06-17    0.180079
2022-06-21    0.174553
2022-06-22    0.168750
2022-06-23    0.163684
2022-06-24    0.185051
2022-06-27    0.170826
2022-06-28    0.192027
2022-06-29    0.169302
2022-06-30    0.175180
2022-07-01    0.183257
2022-07-05    0.186666
2022-07-06    0.158431
2022-07-07    0.162869
2022-07-08    0.161668
2022-07-11    0.172319
2022-07-12    0.145548
2022-07-13    0.159912
2022-07-14    0.184881
2022-07-15    0.165617
2022-07-18    0.154165
2022-07-19    0.168738
2022-07-20    0.146834
2022-07-21    0.158254
2022-07-22    0.170787
dtype: float64


In [ ]:
#save results 
RPS1.to_excel('26_June_22_July-log_reg(baseline)-real_data.xlsx')
# prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('26_June_22_July-PROBABILITIES-log_reg(baseline)-real_data.xlsx')

#####After feature importances
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-06-15    1
2022-06-16    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13442125 0.20663051 0.19024653 0.20351897 0.26518273]
Date
2022-06-15    1
2022-06-16    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16564263 0.21562714 0.1549875  0.23903904 0.2247037 ]
Date
2022-06-15    2
2022-06-16    3
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                ABBV        AEP         AIZ        ALLE        AMAT         AMP  \
Date                                                                            
2022-06-21  147.75  92.066818  168.087662  103.014191  103.575127  244.206192   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-06-21  124.629997  196.050003  175.318207  150.179993  ...  94.955063   

             PYPL          RE       ROL       ROST         UNH         URI  \
Date                                                                         
2022-06-21  82.75  268.616547  36.46323  83.317162  520.825806  279.790009   

                    V        VRSK        XOM  
Date                                          
2022-06-21  215.80777  184.402573  86.905602  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-22  148.470001  93.614655  169.043716  102.944489  101.41066   

                   AMP        AMZN     AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-06-22  243.927475  122.419998  198.25  174.591156  153.009995  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-06-22  94.401566  81.050003  270.306335  36.373466  82.948456   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-06-22  519.779053  274.290009  213.322159  184.472458  86.24205  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                 

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-21  141.850006  104.940002  171.145065  105.991302  104.363106   

                  AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                  ...   
2022-07-21  284.73999  138.229996  214.979996  196.25322  162.240005  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-07-21  102.199997  96.559998  279.808899  36.540001  90.691246   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-21  546.604919  316.690002  212.820007  200.236176  94.080002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-22  140.339996  103.410004  164.163818  102.695572  100.552856   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-07-22  276.98999  133.220001  210.330002  193.584045  157.699997  ...   

                  PRU       PYPL         RE        ROL       ROST         UNH  \
Date                                                                            
2022-07-22  99.110001  93.580002  273.01001  36.400002  87.801399  542.866638   

                   URI           V        VRSK        XOM  
Date                                                       
2022-07-22  307.549988  208.610001  197.900055  94.010002  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-06-17,0.278553,0.112078,0.123989,0.275182,0.203442,0.112428,0.153783,0.071974,0.130173,0.071028,...,0.106516,0.280747,0.134149,0.088433,0.123475,0.075182,0.141097,0.121383,0.123221,0.263127
2022-06-21,0.099204,0.252061,0.106449,0.075941,0.228029,0.068733,0.194940,0.247023,0.067230,0.257108,...,0.106247,0.144765,0.217428,0.090065,0.093188,0.075203,0.203320,0.117241,0.060603,0.158191
2022-06-22,0.098060,0.253804,0.241583,0.103707,0.136124,0.068816,0.198054,0.118211,0.095705,0.129987,...,0.271950,0.207488,0.110508,0.089690,0.222846,0.207423,0.141807,0.071684,0.240243,0.184893
2022-06-23,0.189262,0.252359,0.106353,0.077445,0.136869,0.123295,0.197511,0.119797,0.094143,0.235956,...,0.106725,0.143571,0.065478,0.121941,0.228273,0.100769,0.107756,0.115942,0.126926,0.104349
2022-06-24,0.152638,0.236698,0.123124,0.076753,0.210330,0.244226,0.242261,0.119824,0.294897,0.071787,...,0.062025,0.290134,0.280453,0.240812,0.125257,0.142084,0.237178,0.239800,0.111661,0.104546
2022-06-27,0.286151,0.081332,0.123315,0.102248,0.145381,0.108985,0.107318,0.255082,0.289853,0.108324,...,0.063197,0.131609,0.279685,0.230796,0.093938,0.072760,0.238913,0.071160,0.126254,0.104987
2022-06-28,0.285382,0.082610,0.106487,0.130628,0.119743,0.240085,0.244096,0.073518,0.098782,0.131539,...,0.123771,0.142543,0.065989,0.241337,0.224823,0.140857,0.237123,0.113363,0.061303,0.188277
2022-06-29,0.285280,0.229359,0.124655,0.073533,0.146555,0.259576,0.152140,0.237357,0.066703,0.105169,...,0.065628,0.289991,0.280943,0.230649,0.232293,0.286019,0.125462,0.070319,0.111333,0.266944
2022-06-30,0.097677,0.083617,0.280185,0.221741,0.135875,0.127887,0.125105,0.237273,0.125690,0.105377,...,0.245642,0.287653,0.132769,0.126715,0.233862,0.099604,0.204555,0.242555,0.128064,0.104087


In [ ]:
print(RPS2.mean(axis=1).mean())

0.1586648768037995


In [ ]:
print(RPS2.mean(axis=1))

date
2022-06-17    0.167536
2022-06-21    0.150327
2022-06-22    0.153416
2022-06-23    0.149816
2022-06-24    0.165326
2022-06-27    0.162654
2022-06-28    0.163284
2022-06-29    0.163406
2022-06-30    0.162422
2022-07-01    0.165178
2022-07-05    0.162523
2022-07-06    0.151064
2022-07-07    0.158472
2022-07-08    0.149132
2022-07-11    0.168355
2022-07-12    0.163000
2022-07-13    0.165653
2022-07-14    0.168452
2022-07-15    0.162906
2022-07-18    0.146359
2022-07-19    0.159853
2022-07-20    0.145453
2022-07-21    0.146393
2022-07-22    0.156978
dtype: float64


In [ ]:
#save results 
RPS2.to_excel('26_June_22_July-log_reg(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('26_June_22_July-PROBABILITIES-log_reg(with_feature_imp)-real_data.xlsx')

####**For 24 July-19 August**

#####Features
***

In [ ]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-07-22','2022-08-20') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-07-22  116.639999  117.910004  116.389999  117.540001  112.914551
 2021-07-23  117.980003  118.669998  117.570000  118.190002  113.538971
 2021-07-26  118.129997  118.470001  117.269997  117.790001  113.154709
 2021-07-27  117.790001  118.300003  116.760002  117.959999  113.318008
 2021-07-28  117.629997  118.930000  117.300003  118.550003  113.884796
 ...                ...         ...         ...         ...         ...
 2022-08-15  143.110001  143.279999  140.419998  142.289993  142.289993
 2022-08-16  142.110001  143.979996  142.080002  142.550003  142.550003
 2022-08-17  141.419998  142.759995  140.550003  141.440002  141.440002
 2022-08-18  141.490005  141.580002  139.820007  141.289993  141.289993
 2022-08-19  141.779999  143.360001  141.460007  141.850006  141.850006
 
 [273 rows x 5 columns],
 'AEP':                   Ope

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-07-24') & (stock['ABBV'].index <= '2022-08-19')]))

20


In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-07-22']['Rank'])

Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    2.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    3.0
Name: Rank, dtype: float64
Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    2.0
Name: Rank, dtype: float64
Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    5.0
Name: Rank, dtype: float64
Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    5.0
Name: Rank, dtype: float64
Date
2022-07-22    3.0
Name: Rank, dtype: float64
Date
2022-07-22    3.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    5.0
Name: Rank, dtype: float64
Date
2022-07-22    3.0
Name: Rank, dtype: float64
Date
2022-07-22    5.0
Name: Rank, dtype: float64


#####Logistic Regression (baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-07-11    1.0
2022-07-12    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13565904 0.137871   0.23144419 0.17229441 0.32273137]
Date
2022-07-11    2.0
2022-07-12    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.08189644 0.27815268 0.10218875 0.19213447 0.34562767]
Date
2022-07-11    4.0
2022-07-12    4.0
Name: Rank, dtype: float64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.02911126 0.20722004 0.12234691 0.41468936 0.22663242]
Date
2022-07-11    5.0
2022-07-12    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1928679  0.33437409 0.23584036 0.22623557 0.01068208]
Date
2022-07-11    5.0
2022-07-12    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeI

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Date
2022-07-11    5.0
2022-07-12    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.2014694  0.11946285 0.23877033 0.17311094 0.2671865 ]
Date
2022-07-11    5.0
2022-07-12    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.34622844 0.21490986 0.12869642 0.10689276 0.20327252]
Date
2022-07-11    3.0
2022-07-12    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-16  142.550003

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-07-13,0.310001,0.215822,0.037666,0.234201,0.183910,0.089484,0.158284,0.119367,0.057815,0.079073,...,0.090737,0.153772,0.125547,0.260842,0.097136,0.141773,0.145929,0.215802,0.219690,0.327737
2022-07-14,0.162910,0.109163,0.095913,0.097761,0.232392,0.245054,0.179798,0.053904,0.241121,0.160428,...,0.271485,0.227590,0.120527,0.142469,0.208069,0.075644,0.126568,0.284182,0.373971,0.300260
2022-07-15,0.161003,0.125945,0.284472,0.299871,0.198581,0.120821,0.146484,0.049909,0.274632,0.115777,...,0.069299,0.202579,0.103705,0.228641,0.159689,0.084083,0.130513,0.075801,0.152776,0.331733
2022-07-18,0.071945,0.080373,0.254085,0.107190,0.174200,0.097533,0.220941,0.183248,0.094635,0.139937,...,0.070176,0.127307,0.252223,0.154191,0.093361,0.143984,0.260541,0.089257,0.078871,0.135402
2022-07-19,0.179842,0.151556,0.126036,0.103314,0.236151,0.186814,0.120589,0.343516,0.224665,0.088789,...,0.126555,0.213841,0.300816,0.076955,0.418861,0.095410,0.244022,0.078439,0.100005,0.122530
2022-07-20,0.188520,0.211104,0.084203,0.208432,0.158390,0.143643,0.209165,0.102999,0.063892,0.139758,...,0.102301,0.144609,0.121114,0.142508,0.125937,0.323232,0.108949,0.082173,0.094773,0.078952
2022-07-21,0.091954,0.094398,0.303946,0.213357,0.159842,0.155142,0.214676,0.085671,0.140113,0.154754,...,0.102179,0.136223,0.111352,0.207908,0.401546,0.291641,0.133113,0.088983,0.144876,0.092568
2022-07-22,0.165598,0.063871,0.075491,0.089973,0.322607,0.083121,0.269825,0.260294,0.056477,0.096744,...,0.259740,0.119685,0.123615,0.262275,0.088608,0.208656,0.088896,0.108377,0.261785,0.099123
2022-07-25,0.242834,0.128319,0.089247,0.077149,0.276450,0.080913,0.077033,0.320305,0.121183,0.206880,...,0.060649,0.204323,0.139493,0.269254,0.143258,0.087785,0.146465,0.124220,0.116250,0.081463


In [ ]:
print(RPS1.mean(axis=1).mean())

0.16505242682395396


In [ ]:
print(RPS1.mean(axis=1))

date
2022-07-13    0.161302
2022-07-14    0.187853
2022-07-15    0.165032
2022-07-18    0.148978
2022-07-19    0.167850
2022-07-20    0.144573
2022-07-21    0.157063
2022-07-22    0.159043
2022-07-25    0.167842
2022-07-26    0.175013
2022-07-27    0.155198
2022-07-28    0.151056
2022-07-29    0.167504
2022-08-01    0.165434
2022-08-02    0.167886
2022-08-03    0.152746
2022-08-04    0.178129
2022-08-05    0.171023
2022-08-08    0.168489
2022-08-09    0.179396
2022-08-10    0.161452
2022-08-11    0.152238
2022-08-12    0.164446
2022-08-15    0.191711
dtype: float64


In [ ]:
#save results 
#save results 
RPS1.to_excel('24_July_19_August-log_reg(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('24_July_19_August-PROBABILITIES-log_reg(baseline)-real_data.xlsx')

#####After feature importances
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-07-11    1.0
2022-07-12    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15909045 0.20964108 0.17482993 0.20004001 0.25639854]
Date
2022-07-11    2.0
2022-07-12    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-15'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16500348 0.20653096 0.194493   0.22228747 0.21168508]
Date
2022-07-11    4.0
2022-07-12    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-13'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-14  142.600006  103.779999  165.597916  105.951485  110.098442   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-14  289.170013  143.550003  218.160004  201.651291  165.839996  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-14  104.265739  101.099998  273.894653  36.970001  89.804367   

                   UNH         URI           V        VRSK   XOM  
Date                                                              
2022-07-14  541.999329  336.549988  211.330002  202.532333  94.0  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-15  142.289993  104.559998  166.573883  106.738075  109.499977   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-15  286.339996  143.179993  218.380005  200.366501  165.399994  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-15  103.603516  101.510002  273.487122  36.900002  89.844223   

                   UNH         URI           V        VRSK    XOM  
Date                                                               
2022-07-15  542.936401  337.519989  216.419998  205.327682  92.32  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-16  142.550003

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-07-13,0.275757,0.216675,0.064358,0.278752,0.218200,0.106195,0.149706,0.118051,0.065335,0.072949,...,0.120846,0.137304,0.124959,0.231047,0.117925,0.112283,0.126503,0.238254,0.240182,0.263612
2022-07-14,0.134710,0.096836,0.102321,0.126453,0.212515,0.259310,0.126159,0.071271,0.249862,0.107463,...,0.243685,0.204305,0.125395,0.123260,0.132293,0.070982,0.110005,0.259054,0.268590,0.268671
2022-07-15,0.135410,0.136948,0.236723,0.277461,0.207055,0.109356,0.149230,0.071302,0.260631,0.108246,...,0.069407,0.206096,0.113709,0.237502,0.223761,0.071175,0.110011,0.069767,0.240587,0.263979
2022-07-18,0.083129,0.096688,0.236879,0.126996,0.205467,0.107282,0.202515,0.259988,0.117256,0.108248,...,0.069123,0.131281,0.226599,0.125557,0.119714,0.114492,0.213201,0.105978,0.061496,0.179439
2022-07-19,0.134407,0.137957,0.123031,0.126522,0.222344,0.243313,0.109372,0.238049,0.250362,0.128024,...,0.121862,0.213733,0.225771,0.082341,0.229916,0.128490,0.213683,0.069798,0.061706,0.176064
2022-07-20,0.207648,0.215952,0.105969,0.225390,0.208305,0.129709,0.199587,0.071434,0.065192,0.107800,...,0.117313,0.132987,0.112144,0.127799,0.093758,0.226287,0.126892,0.069952,0.113645,0.106723
2022-07-21,0.106443,0.097786,0.271062,0.225799,0.207291,0.129790,0.200214,0.071250,0.116180,0.108310,...,0.117756,0.131426,0.130410,0.232723,0.222170,0.229343,0.128727,0.070041,0.123480,0.174966
2022-07-22,0.137488,0.080169,0.069992,0.106565,0.220169,0.104072,0.241395,0.240040,0.066176,0.126475,...,0.243885,0.142893,0.112400,0.244051,0.130764,0.257159,0.132630,0.110700,0.240910,0.175338
2022-07-25,0.275951,0.136991,0.106300,0.106665,0.218324,0.068055,0.108101,0.253913,0.110515,0.254033,...,0.071343,0.205943,0.112675,0.242591,0.249259,0.070456,0.109868,0.111856,0.112767,0.105343


In [ ]:
print(RPS2.mean(axis=1).mean())

0.15833122412067915


In [ ]:
print(RPS2.mean(axis=1))

date
2022-07-13    0.165338
2022-07-14    0.167862
2022-07-15    0.161319
2022-07-18    0.146742
2022-07-19    0.159896
2022-07-20    0.145490
2022-07-21    0.146821
2022-07-22    0.160689
2022-07-25    0.163233
2022-07-26    0.165854
2022-07-27    0.148690
2022-07-28    0.157862
2022-07-29    0.161148
2022-08-01    0.163903
2022-08-02    0.151565
2022-08-03    0.155666
2022-08-04    0.158660
2022-08-05    0.164287
2022-08-08    0.160074
2022-08-09    0.160855
2022-08-10    0.161329
2022-08-11    0.146303
2022-08-12    0.160642
2022-08-15    0.165722
dtype: float64


In [ ]:
#save results 
RPS2.to_excel('24_July_19_August-log_reg(with_feature_imp)-one_m_less.xlsx')
# prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('24_July_19_August-PROBABILITIES-log_reg(with_feature_imp)-one_m_less.xlsx')